# Création du Dataframe + initialisation de la Data

Pensez a gerer les colonnes en trop excel (df.drop) \
Pour chimiste 2 mso trop long penser a tout mettre dans 1 case \
A faire je pense gestion du profile du gars du type bouton `Mon Profil` avec possibilité de changer année groupe filiere et Mso

In [115]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
Df = pd.read_csv('./planning_Csv/4ETI/planning2003.csv', sep=';')
# Filter columns and rows
Df.reset_index()
Df.fillna(0, inplace=True)
# on ne garde que les 5 premieres columns
Df = Df.iloc[:, 0:5]
Df

,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI
0,20-mars,21-mars,22-mars,23-mars,24-mars
1,8h-12h15,8h-12h15,8h-12h15,8h-12h15,8h-12h15
2,C ONCEP.LOGICIELLE/BIG DATA,CONCEP.LOGICIELLE/BIG DATA,GROUPE A & B,GROUPE C,CONCEP.LOGICIELLE/BIG DATA
3,Data Mining,TIDAL,Projet,Projet,Administration Système
4,0,Y.JOLY,0,0,G.MOREL
5,J.SAMUEL\nSalles G103 + I300\n10h30-12h30,Salle I203,F. JOLY,F. JOLY,Salles I300 +I306
6,0,0,0,0,10h30 - 12h30
7,ROBOTIQUE,ROBOTIQUE,Salle I105 - Groupe A,Salle I105,ROBOTIQUE
8,Data Mining,TIDAL,Salle I200 - Groupe B,0,Modélisation des Robots
9,0,Y.JOLY,0,0,R.LEBER


### On récupère la liste de toutes les matière/prof/majeurs dans LstSemaine

In [116]:
def createSchedule(Df):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case.strip())

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30','13h30 - 17h45','13h30 - 17h30'} :
                  IsAprem = True
                elif value in {'8h-12h15','8h - 12h15', '8h30 - 10h00', '8h-10h', '10h15-12h15','10h15 - 12h15', '8h- 10h'}:
                    IsAprem = False
                if "\n" in value :
                    # on ajoute les cours
                    if IsAprem :
                        LstAprem.append(value.split("\n")[0])
                        LstAprem.append(value.split("\n")[1])
                    else :
                        LstMatin.append(value.split("\n")[0])
                        LstMatin.append(value.split("\n")[1])
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
#df_semaine.to_json('planning_semaineMatin_Aprem_TEST.json',orient='records', indent=4)

,jour,Matin,Aprem
0,LUNDI-20-mars,"[8h-12h15, C ONCEP.LOGICIELLE/BIG DATA, Data M...","[13h10-18h, LV, (Voir planning), LV\n(Voir pla..."
1,MARDI-21-mars,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, TIDAL, ...","[13h30-17h45, GROUPE C&D, Projet, F. JOLY, Sal..."
2,MERCREDI-22-mars,"[8h-12h15, GROUPE A & B, Projet, F. JOLY, Sall...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Pas ..."
3,JEUDI-23-mars,"[8h-12h15, GROUPE C, Projet, F. JOLY, Salle I1...",[13h30-17h45]
4,VENDREDI-24-mars,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Adminis...","[13h30-17h45, MSO1 OUVERTURE, AWS Cloud Founda..."


### On filtre les Majeurs et les GROUPES par jour/ Demi-journée et majeurs ou tronc commun 4ETI

In [ ]:
import copy
def filtreMajGrp(pLstSemaine):
    Semaine =  {}
    lstGroupesinit = ['A','B','C','D','TousLesGroupes']
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']

    # creation dico {grp: [], ...}
    dicGrpinit = {}
    for grp in lstGroupesinit :
        dicGrpinit[grp] = []
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit :
        dicMajinit[maj] = copy.deepcopy(dicGrpinit)
    
    print(f'{dicMajinit=}')

    print(f"test: {id(dicMajinit['ROBOTIQUE']['A'])=}", f"and {id(dicMajinit['IMAGE']['A'])=}")

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        grp = "TousLesGroupes" #on initialise le groupe à tous
        # creation de la journee ("LUNDI-..." : {})
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        for Case in jour["Aprem"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeurGrp = filtreMajGrp(copy.deepcopy(LstSemaine)) # KK
df_planningMajGrp = pd.DataFrame(PlanningMajeurGrp)
df_planningMajGrp

## On filtres les majeurs de chaque jours

In [117]:
import copy
def filtreMaj(pLstSemaine):
    Semaine =  {}
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit:
        dicMajinit[maj] = []    

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        Majeur = "Pour tous" #on initialise le majeur à tous
        for Case in jour["Aprem"]:
            if Case in lstMajeursinit:
                Majeur = Case
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeur = filtreMaj(copy.deepcopy(LstSemaine)) # KK
df_planningMaj = pd.DataFrame(PlanningMajeur)
df_planningMaj

jour["jour"]='LUNDI-20-mars'
jour["jour"]='MARDI-21-mars'
jour["jour"]='MERCREDI-22-mars'
jour["jour"]='JEUDI-23-mars'
jour["jour"]='VENDREDI-24-mars'
fin


,LUNDI-20-mars,MARDI-21-mars,MERCREDI-22-mars,JEUDI-23-mars,VENDREDI-24-mars
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'..."


### Gestion des 4CGP

In [ ]:
def filtre4CGP(LstSemaine) : # KK
    Semaine =  {}
    for jour in LstSemaine :
        print(jour["jour"])
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {
            "Groupe A":[],
            "Groupe B":[],
            "Groupe C":[],
            "Chimie Médicinale et Hétérocycles":[],
            "Génie de la réaction chimique avancée":[],
            "Transition Energétique":[],
            "Analyse en ligne" : [],
            "Synthèse Macromoléculaire":[],
            "Pour tous":[]
        } 

filtre4CGP(LstSemaine)

In [ ]:
with open('./Output_Json/Datatest.json', 'w+') as f:
    json.dump(PlanningMajeur, f, indent=4, cls=NumpyEncoder)

